In [120]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [121]:
Test= pd.read_csv('../input/bigmart-sales-data/Test.csv')
Test.shape


In [122]:

Train=pd.read_csv('../input/bigmart-sales-data/Train.csv')
Train.shape


In [123]:

data=pd.concat([Train,Test],ignore_index=True)
data.columns


In [124]:

sns.distplot(data['Item_Outlet_Sales'])
print('skewnes:',data['Item_Outlet_Sales'].skew())
print('kurtosis:',data['Item_Outlet_Sales'].kurt())


In [125]:

data.isna().sum()


In [126]:

data=data.fillna({'Item_Weight':  data['Item_Weight'].mean(),'Outlet_Size':data.loc[:,'Outlet_Size'].value_counts().index[0],
          'Item_Outlet_Sales':0   })
data.isna().sum()


In [127]:
categorical_data= data.select_dtypes(include=[object])
categorical_data.columns


In [128]:
continuous_data=data.select_dtypes(include=[np.float64]+[np.int64])

continuous_data.columns

In [129]:
data.iloc[:,[2]].value_counts()
data.iloc[:,2]=data.iloc[:,[2]].replace({'LF':'Low Fat','low fat' :'Low Fat'  })
data.iloc[:,[2]].value_counts()


In [130]:
data['itemidentifernew']=data.iloc[:,0].str[0:2]
data['itemidentifernew'].value_counts()

In [131]:
plt.figure
sns.set(style='ticks')
sns.pairplot(continuous_data )

In [132]:
datan=data.drop(['Outlet_Establishment_Year','Item_Identifier'],axis=1)
X=datan.values
X[0]

In [133]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(sparse=False,drop='first'),[3,5,10])
,('OrdinalEncoder',OrdinalEncoder(categories='auto'),[1,6,7,8])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)

X=ct.fit_transform(X) 

X.shape
Xnew=X[:,0:34]
Ynew=X[:,-1]
Xnew.shape,Ynew.shape

In [134]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xnew,Ynew,test_size=.2,random_state=42)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)


In [135]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score

regr = linear_model.LinearRegression()
k=regr.fit(X_train,y_train)



In [136]:
y_pred=regr.predict(X_test)

In [137]:
k.score(X_train,y_train)

In [138]:
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error
r2_score(y_test,y_pred)


In [139]:
plt.scatter(y_test,y_pred)